In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Titanic").getOrCreate()

In [4]:
df = spark.read.csv("titanic.csv", inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = df.select(['Survived',
 'Pclass', 'Sex',
 'Age',
 'SibSp',
 'Parch', 'Fare',
 'Cabin',
 'Embarked'])

In [46]:
my_final_data = my_cols.na.drop()

In [36]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer, OneHotEncoder

In [47]:
gender_index = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVect')

In [56]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVect')

In [57]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVect', 'EmbarkVect', 'Age', 'SibSp', 'Parch', 'Fare'], 
                            outputCol="features")

In [58]:
from pyspark.ml.classification import LogisticRegression

In [59]:
from pyspark.ml import Pipeline

In [60]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [61]:
pipeline = Pipeline(stages=[gender_index, embark_indexer, gender_encoder, embark_encoder, assembler,
                            log_reg_titanic])

In [62]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [63]:
fit_model = pipeline.fit(train_data)

In [64]:
results= fit_model.transform(test_data)

In [73]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [82]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [83]:
results.select(['Survived', 'prediction']).show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
|       1|       1.0|
+--------+----------+
only showing top 20 rows



In [84]:
AUC = my_eval.evaluate(results)

In [85]:
AUC

0.640625